In [0]:
%pip install -qqqr ../requirements-dev.txt
%pip install -qqq --upgrade databricks-sdk
dbutils.library.restartPython()

In [0]:
%python


import sys
sys.path.append("./src")


In [0]:
metadata_table: str = "nfleming.harbor_freight.control_table"
volume: str = "/Volumes/nfleming/harbor_freight/data"
config_path: str = "/Workspace/Users/nate.fleming@databricks.com/.bundle/delta_live_store/dev/files/src/sample.yml"
group: str = "main.manufacturing"

spark.conf.set("dls.metadata_table", metadata_table)
spark.conf.set("dls.volume", volume)
spark.conf.set("dls.config_path", config_path)
spark.conf.set("dls.group", group)

In [0]:
from dbrx.dls.store import DeltaLiveStore, DeltaLiveEntity, DeltaLiveEntityList
from dbrx.dls.filters import *
from dbrx.dls.loaders import DeltaLiveStoreLoader, YamlLoader, FileServiceLoader, entity_from_directory_entry, medallion_from_directory_entry

store: DeltaLiveStore = DeltaLiveStore(table=metadata_table)

In [0]:

store.destroy()
store.initialize()

In [0]:
%sql

describe identifier("${dls.metadata_table}")

In [0]:
%sql

select * from identifier("${dls.metadata_table}")

In [0]:
import pyspark.sql.functions as F

# Generate a python list with 30 different names
names = ['dataset1', 'dataset2', 'dataset3', 'dataset4', 'dataset5', 'dataset6', 'dataset7', 'dataset8', 'dataset9', 'dataset10',
         'dataset11', 'dataset12', 'dataset13', 'dataset14', 'dataset15', 'dataset16', 'dataset17', 'dataset18', 'dataset19', 'dataset20',
         'dataset21', 'dataset22', 'dataset23', 'dataset24', 'dataset25', 'dataset26', 'dataset27', 'dataset28', 'dataset29', 'dataset30']

src_df = spark.read.option("header", "true").option("inferSchema", "true").csv("/Volumes/nfleming/harbor_freight/data/high_temps")
for name in names:
  src_df.withColumn("id", F.monotonically_increasing_id()).write.mode("overwrite").option("header", "true").csv(f"{volume}/{name}")
  

In [0]:
%sql

LIST "${dls.volume}"

In [0]:


host: str = spark.conf.get("spark.databricks.workspaceUrl", None)
token: str = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
loader: DeltaLiveStoreLoader = FileServiceLoader(
  host=host, 
  token=token, 
  path=volume, 
  factory=entity_from_directory_entry(group=group))
loader.load(store)

In [0]:

loader: YamlLoader = YamlLoader(
  path=config_path
)
loader.load(store)


In [0]:

display(store.find_entities(by_is_enabled() & by_is_latest()).to_df())

In [0]:
%sql

select entity_id, expectations, is_quarantined, is_streaming, is_enabled from identifier("${dls.metadata_table}") where is_latest;

In [0]:
%sql

select * from identifier("${dls.metadata_table}")

In [0]:
store.destroy()

In [0]:
from dbrx.dls.filters import by_is_enabled

display(store.find_entity(by_is_enabled()))

In [0]:
from dbrx.dls.filters import *

display(store.find_entities(by_is_enabled() & by_is_latest() & by_group(group)).to_df())
